# Import Library

In [4]:
import os
from dotenv import load_dotenv

# Vector DB
from qdrant_client import QdrantClient

# Embedding Model
from langchain_ollama import OllamaEmbeddings

# Large Language Model
from langchain_ollama.llms import OllamaLLM
from langchain_huggingface import HuggingFaceEndpoint, HuggingFacePipeline
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
# from langchain.prompts import ChatPromptTemplate
# import transformers
# import torch

# Document Retrieval

### Embedding Model

In [2]:
def get_embedding_model(model_name):
    '''
    Args:
        model_name (str): Name of the model to be used for embeddings
    
    Returns:
        embeddings: Embedding model object
    '''

    embeddings = OllamaEmbeddings(
        model=model_name
    )
    return embeddings

def encode_text(embeddings, text):
    '''
    Args:
        embeddings (OllamaEmbeddings): Embedding model object
        text (str): Text to be encoded

    Returns:
        np.array: Encoded vector for the input text
    '''
    
    return embeddings.embed_query(text)

### Retrieval

In [3]:
def get_chunks(vector, client, collection_name):
    '''
    Args:
        query (str): Query text
        embeddings (OllamaEmbeddings): Embedding model object
        client (QdrantClient): Qdrant client object
        collection_name (str): Name of the collection to search for chunks

    Returns:
        chunks (list): list of top 5 chunks from the collection
    '''
    
    # Mengambil 5 chunk teratas dengan similarity tertinggi
    chunks = client.query_points(
        collection_name=collection_name,
        query=vector,
        limit=5
    )
    chunks = chunks.points
    return chunks

def get_content_list(chunks):
    '''
    Args:
        chunks (list): List of ScoredPoint
    Returns:
        list: List of content
    '''

    # Mengambil isi dari setiap chunk
    return [chunk.payload['text'].replace('Â','') for chunk in chunks]

def get_source_list(chunks):
    '''
    Args:
        chunks (list): List of ScoredPoint
    Returns:
        list: List of sources
    '''

    # Mengambil source dari setiap chunk
    return [chunk.payload['metadata']['source'] for chunk in chunks]

In [6]:
# Menguji pengambilan chunks dari Qdrant

load_dotenv()
client = QdrantClient(url=os.getenv("QDRANT_URL"), api_key=os.getenv("QDRANT_API_KEY"))
collection_name = "cvd_collection_v1"
embeddings = get_embedding_model('nomic-embed-text')

query = "Bagaimana cara mencegah penyakit hipertensi?"
query_vector = encode_text(embeddings, query)

chunks = get_chunks(query_vector, client, collection_name)
content_list = get_content_list(chunks)

for i, content in enumerate(content_list):
    print(f"Chunk {i+1}: {content}")

Chunk 1: Topik: Hipertensi, Subtopik: Pengobatan Hipertensi 
 Tekanan darah tinggi bisa diatasi dengan mengubah gaya hidup menjadi lebih sehat. Namun, pada beberapa penderita, perubahan gaya hidup juga harus disertai dengan konsumsi obat antihipertensi.
Perlu atau tidaknya penggunaan obat antihipertensi tergantung pada nilai tekanan darah pasien dan seberapa besar risiko pasien terserang komplikasi, seperti stroke atau serangan jantung.
Berikut ini adalah beberapa metode pengobatan yang dapat digunakan untuk menangani hipertensi:

Chunk 2: Topik: Hipertensi, Subtopik: Pengobatan Hipertensi: Perubahan gaya hidup 
 Mengubah gaya hidup menjadi lebih sehat bisa menurunkan tekanan darah dalam beberapa minggu. Biasanya, dokter akan menyarankan perubahan gaya hidup tanpa perlu konsumsi obat jika risiko pasien terserang komplikasi rendah.
Gaya hidup sehat yang dijalani adalah:
- Mengonsumsi lebih banyak buah-buahan dan sayur-sayuran
- Mengurangi konsumsi garam, yaitu maksimal sebanyak satu sen

# Generating Response

### Formulating a Prompt

In [8]:
def generate_prompt(content_list, query):
    '''
    Args:
        content_list (list): List of content
    Returns:
        str: Prompt for LLM
    '''

    # Membuat prompt untuk LLM menggunakan konteks yang diberikan
    prompt = "Diberikan konteks berikut:\n"
    for num, content in enumerate(content_list):
        prompt += f"{content}"
        prompt += "------------------\n"
    # prompt += "Jawab pertanyaan ini sesuai dengan konteks yang diberikan: " + query
    prompt += "Berdasarkan konteks yang diberikan, jawab pertanyaan ini: " + query
    prompt += "\nJawab pertanyaan tanpa menyebutkan 'berdasarkan konteks yang diberikan'"
    return prompt

def generate_hyde_prompt(query):
    '''
    Args:
        query (str): Query text
    Returns:
        str: Prompt for LLM
    '''

    # Membuat prompt untuk LLM untuk pipeline RAG + HyDE
    # Prompt ini digunakan agar LLM menjawab pertanyaan dengan pengetahuan yang dimiliki
    prompt = "Kamu adalah seorang ahli penyakit jantung. Jawab pertanyaan ini dengan pengetahuan yang kamu miliki: " + query
    return prompt

In [10]:
# Menguji pembuatan prompt untuk LLM
prompt = generate_prompt(content_list, query)

print(prompt)

Diberikan konteks berikut:
Topik: Hipertensi, Subtopik: Pengobatan Hipertensi 
 Tekanan darah tinggi bisa diatasi dengan mengubah gaya hidup menjadi lebih sehat. Namun, pada beberapa penderita, perubahan gaya hidup juga harus disertai dengan konsumsi obat antihipertensi.
Perlu atau tidaknya penggunaan obat antihipertensi tergantung pada nilai tekanan darah pasien dan seberapa besar risiko pasien terserang komplikasi, seperti stroke atau serangan jantung.
Berikut ini adalah beberapa metode pengobatan yang dapat digunakan untuk menangani hipertensi:
------------------
Topik: Hipertensi, Subtopik: Pengobatan Hipertensi: Perubahan gaya hidup 
 Mengubah gaya hidup menjadi lebih sehat bisa menurunkan tekanan darah dalam beberapa minggu. Biasanya, dokter akan menyarankan perubahan gaya hidup tanpa perlu konsumsi obat jika risiko pasien terserang komplikasi rendah.
Gaya hidup sehat yang dijalani adalah:
- Mengonsumsi lebih banyak buah-buahan dan sayur-sayuran
- Mengurangi konsumsi garam, yaitu

### Generating LLM Response

In [ ]:
def get_llm(model_name, from_ollama=True, run_locally=True):
    '''
    Args:
        model_name (str): Name of the model to be used for LLM
        from_ollama (bool): Whether the model is from Ollama or Hugging Face
    Returns:
        llm: LLM model object
    '''

    # Mengambil model LLM dari Ollama atau Hugging Face
    if from_ollama:
        llm = OllamaLLM(model=model_name)
    else:
        if run_locally:
            # model_id = model_name
            # tokenizer = AutoTokenizer.from_pretrained(model_id)
            # model = AutoModelForCausalLM.from_pretrained(model_id)
            # pipe = pipeline(
            #     "text-generation", 
            #     model=model, 
            #     tokenizer=tokenizer, 
            #     max_new_tokens=512
            # )
            # llm = HuggingFacePipeline(pipeline=pipe)
            llm = HuggingFacePipeline.from_model_id(
                model_id=model_name,
                task="text-generation",
                pipeline_kwargs={"max_new_tokens": 512}
            )
        else:
            llm = HuggingFaceEndpoint(
                repo_id=model_name,
                huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
                max_new_tokens=512
            )
    return llm

def get_response(model, prompt):
    '''
    Args:
        model (OllamaLLM/HuggingFaceEndpoint): LLM model object
        prompt (str): Prompt for the user
    Returns:
        reponse (str): Response from the LLM model
    '''

    # Mengambil response dari LLM model
    response = model.invoke(prompt).strip()
    return response


# Retrieval and Generation Pipeline

### Vanilla RAG

In [ ]:
def retrieval_pipeline(url, api_key, collection_name, embedding_model_name, query, return_sources=False):
    '''
    Args:
        url (str): URL of the Qdrant server
        api_key (str): API key for Qdrant
        collection_name (str): Name of the collection to search for chunks
        embedding_model_name (str): Name of the model to be used for embeddings
        query (str): Query text
        return_sources (bool): Whether to return sources along with content
    Returns:
        content_list (list): List of content
        (optional) source_list (list): List of sources
    '''

    # Get Qdrant client
    client = QdrantClient(url=url, api_key=api_key)

    # Mengambil embedding model
    embeddings = get_embedding_model(embedding_model_name)

    # Mengubah teks query menjadi vector embedding
    query_vector = encode_text(embeddings, query)

    # Mengambil 5 chunk teratas dari Qdrant
    chunks = get_chunks(query_vector, client, collection_name)

    # Mengambil isi dan source dari setiap chunk
    content_list = get_content_list(chunks)
    source_list = get_source_list(chunks)

    if return_sources:
        return content_list, source_list
    else:   
        return content_list

def generation_pipeline(content_list, query, llm_name, from_ollama=True, run_locally=False):
    '''
    Args:
        content_list (list): List of content
        query (str): Query text
        llm_name (str): Name of the model to be used for LLM
        from_ollama (bool): Whether the model is from Ollama or Hugging Face
        run_locally (bool): Whether to run the model locally or on Hugging Face
    Returns:
        str: Response from the LLM model
    '''

    # Membuat prompt untuk LLM
    prompt = generate_prompt(content_list, query)

    # Mengambil model LLM dari Ollama atau Hugging Face
    llm = get_llm(llm_name, from_ollama, run_locally)

    # Mengambil response dari LLM model
    response = get_response(llm, prompt)
    return response

### RAG + HyDE Pipeline

In [14]:
def hyde_retrieval_pipeline(url, api_key, collection_name, embedding_model_name, hyde_model_name, query, from_ollama=True, run_locally=False, return_sources=False):
    '''
    Args:
        url (str): URL of the Qdrant server
        api_key (str): API key for Qdrant
        collection_name (str): Name of the collection to search for chunks
        embedding_model_name (str): Name of the model to be used for embeddings
        hyde_model_name (str): Name of the model to be used for HyDE
        query (str): Query text
        from_ollama (bool): Whether the model is from Ollama or Hugging Face
        run_locally (bool): Whether to run the model locally or on Hugging Face
        return_sources (bool): Whether to return sources along with content
    Returns:
        list: List of content
    '''

    # Get Qdrant client
    client = QdrantClient(url=url, api_key=api_key)

    # Mengambil embedding model
    embeddings = get_embedding_model(embedding_model_name)

    # Mengambil model LLM untuk menghasilkan jawaban hipotesis
    hyde_llm = get_llm(hyde_model_name, from_ollama, run_locally)

    # Menghasilkan prompt untuk LLM HyDE
    hyde_prompt = generate_hyde_prompt(query)

    # Mengambil response dari LLM HyDE
    hyde_response = get_response(hyde_llm, hyde_prompt)

    # Mengubah teks jawaban hipotesis menjadi vector embedding
    query_vector = encode_text(embeddings, hyde_response)

    # Mengambil 5 chunk teratas dari Qdrant
    chunks = get_chunks(query_vector, client, collection_name)

    # Mengambil isi dan source dari setiap chunk
    content_list = get_content_list(chunks)
    source_list = get_source_list(chunks)

    if return_sources:
        return content_list, source_list
    else:   
        return content_list

# Testing Pipeline

In [16]:
url = os.getenv("QDRANT_URL")
api_key = os.getenv("QDRANT_API_KEY")
collection_name = "cvd_collection_v1"

embedding_model_name = "nomic-embed-text"
llm_name = "llama3.1"

query = "Apa saja gejala hipertensi?"

### Vanilla RAG

In [17]:
vanilla_rag_content_list = retrieval_pipeline(url, api_key, collection_name, embedding_model_name, query)

print(query)
for content in vanilla_rag_content_list:
    print(content)

Apa saja gejala hipertensi?
Topik: Hipertensi, Subtopik: Gejala Hipertensi 
 Hipertensi merupakan penyakit yang berbahaya, karena bisa terjadi tanpa gejala. Bahkan, pada beberapa kasus, gejalanya baru muncul setelah hipertensi makin parah dan sampai mengancam nyawa. Gejala yang dapat muncul pada kondisi tersebut adalah:
- Mual dan muntah
- Sakit kepala
- Mimisan
- Sesak napas
- Nyeri dada
- Gangguan penglihatan
- Telinga berdenging
- Gangguan irama jantung
- Darah dalam urine

Topik: Hipertensi, Subtopik: Pengobatan Hipertensi 
 Tekanan darah tinggi bisa diatasi dengan mengubah gaya hidup menjadi lebih sehat. Namun, pada beberapa penderita, perubahan gaya hidup juga harus disertai dengan konsumsi obat antihipertensi.
Perlu atau tidaknya penggunaan obat antihipertensi tergantung pada nilai tekanan darah pasien dan seberapa besar risiko pasien terserang komplikasi, seperti stroke atau serangan jantung.
Berikut ini adalah beberapa metode pengobatan yang dapat digunakan untuk menangani hip

In [18]:
vanilla_rag_llm_response = generation_pipeline(vanilla_rag_content_list, query, llm_name)

print(vanilla_rag_llm_response)

Gejala hipertensi antara lain:
- Mual dan muntah
- Sakit kepala
- Mimisan
- Sesak napas
- Nyeri dada
- Gangguan penglihatan
- Telinga berdenging
- Gangguan irama jantung
- Darah dalam urine


### RAG + HyDE Pipeline

In [19]:
hyde_rag_content_list = hyde_retrieval_pipeline(url, api_key, collection_name, embedding_model_name, llm_name, query)

print(query)
for content in hyde_rag_content_list:
    print(content)

Apa saja gejala hipertensi?
Topik: Hipertensi, Subtopik: Pengertian Hipertensi 
 Hipertensi atau darah tinggi adalah kondisi ketika tekanan darah berada pada angka 130/80 mmHg atau lebih. Jika tidak segera ditangani, hipertensi bisa menyebabkan komplikasi serius, seperti gagal jantung, penyakit ginjal, hingga stroke.
Tekanan darah dinyatakan dalam dua nilai angka yang dipisahkan dengan garis miring atau yang biasanya disebut  per  . Angka di awal, yaitu di sebelah kiri garis miring menandakan tekanan sistolik. Ini adalah tekanan di dalam pembuluh darah ketika jantung berkontraksi untuk memompa darah keluar dari jantung.

Angka di akhir yang berada setelah garis miring menandakan tekanan diastolik, yaitu tekanan darah saat jantung berelaksasi dan menyedot atau menerima darah masuk kembali ke dalam jantung.
Pada kondisi normal, tekanan darah orang dewasa adalah 120/80 mmHg. Artinya, tekanan sistoliknya adalah 120 mmHg dan diastoliknya 80 mmHG.
Tekanan darah tinggi yang terjadi terus-mene

In [20]:
hyde_rag_llm_response = generation_pipeline(hyde_rag_content_list, query, llm_name)

print(hyde_rag_llm_response)

Gejala hipertensi adalah:
- Sesak napas
- Nyeri dada
- Sulit berbicara
- Sakit kepala yang parah
- Mati rasa
- Lemas
- Gangguan penglihatan.
